In [3]:
import catboost
import pandas as pd
import numpy as np
from typing import List
import pandas as pd
from scipy import stats
import datetime
import rpy2
from functools import reduce
from catboost import CatBoostRegressor
import time
import seaborn as sns
import ast
import numpy as np
from dateutil import parser
from kshape.core import kshape, zscore
from tslearn.clustering import KShape
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import catboost

from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
import time
import pickle
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Path fo import files
#input data
import_frontenddata = "import/startup/frontendData.csv"
import_orderdata = "import/startup/orderdata.csv"
import_userdata = "import/startup/userdata.csv"
#generated data
transformed_analyzed_events = 'export/startup/time_series_events.csv'
transformed_analyzed_amount = 'export/startup/time_series_amount.csv'

monetary_time_series = "export/startup/startup_monetary_2000.csv"
frequency_time_series = "export/startup/startup_frequency_2000.csv"
recency_time_series = "export/startup/startup_recency_2000.csv"
cluster_centroids_time_series = "export/startup/startup_cluster_centroids_2000.csv"
#prediction
prediction_path = "export/startup/startup_prediction.csv"

## 0. Preprocessing data from startup

In [5]:
df_frontendata = pd.read_csv(import_frontenddata).drop(columns =['manufacturer','model','operatingSysterm','operatingSystemVersion','timezone','event'])
df_orderdata = pd.read_csv(import_orderdata).set_index("Unnamed: 0").drop(columns = ['supplierId','orderSupplierStatus'])
df_userdata = pd.read_csv(import_userdata).set_index("Unnamed: 0")

In [6]:
df_frontendata['timestamp'] = df_frontendata.dropna()["timestamp"].apply(lambda x:datetime.datetime.strptime(str(x)[:10],"%Y-%m-%d"))
df_frontendata['event']= 1
df_frontendata = df_frontendata.groupby(by=['userid','timestamp']).sum().reset_index()
# If we have several actions per day - it is still 1 event
df_frontendata['event']= 1
df_frontendata.head()

,userid,timestamp,event
0,0043a6cb-6314-4844-bd20-06c41915d398,2018-11-30,1
1,0043a6cb-6314-4844-bd20-06c41915d398,2018-12-03,1
2,0043a6cb-6314-4844-bd20-06c41915d398,2019-01-25,1
3,0043a6cb-6314-4844-bd20-06c41915d398,2019-02-07,1
4,0043a6cb-6314-4844-bd20-06c41915d398,2019-02-13,1


In [7]:
# Extract total amount from list of products
def products_to_amount(s):
    amount = 0 
    #print(s)
    for x in ast.literal_eval(s):
        if 'amount' in x:
            amount += int(x['amount'])
    return amount

In [8]:
df_orderdata['createdAt'] = df_orderdata.dropna()["createdAt"].apply(lambda x:datetime.datetime.utcfromtimestamp(int(str(x)[:10])))
df_orderdata['createdAt'] = df_orderdata.dropna()["createdAt"].apply(lambda x:datetime.datetime.strptime(str(x)[:10],"%Y-%m-%d"))
df_orderdata['products'] = df_orderdata.dropna()["products"].apply(lambda x:products_to_amount(x))
df_orderdata.columns = ['timestamp','amount','userid']
df_orderdata= df_orderdata.reset_index().drop(columns = ['Unnamed: 0'])
df_orderdata = df_orderdata.groupby(by=['userid','timestamp']).sum().reset_index()
df_orderdata.head()

,userid,timestamp,amount
0,0001c752-8daf-424d-af34-574f3ba624bb,2018-08-02,19.0
1,0001c752-8daf-424d-af34-574f3ba624bb,2018-08-16,8.0
2,0001c752-8daf-424d-af34-574f3ba624bb,2018-08-23,19.0
3,0001c752-8daf-424d-af34-574f3ba624bb,2018-08-30,16.0
4,00653016-d177-4080-86e6-4c69b865cb02,2019-01-27,147.0


In [9]:
df_front_back_combined = df_frontendata.merge(df_orderdata,how = 'outer',left_on =['timestamp','userid'],right_on=['timestamp','userid']).dropna(subset=['timestamp', 'userid'])
#df_front_back_combined=df_front_back_combined.groupby(by=['userid','timestamp']).sum().reset_index()
# For Monetary only amount is needed
# We count for event if either purchase or action took place
df_front_back_combined['event'] = df_front_back_combined.apply((lambda x: 1 if (np.isnan(x['amount']) or np.isnan(x['event'])) else 2) ,axis =1 )
df_front_back_combined=df_front_back_combined.groupby(by=['userid','timestamp']).sum().reset_index()
df_front_back_combined.sample(5)

,userid,timestamp,event,amount
6648,3a4ab70c-9f8d-4398-8b89-6c55f0808768,2019-02-07,1,0.0
23052,acc897ee-392b-4456-a720-e0efafd65879,2019-01-16,2,93.0
27129,ce372759-85c4-40b9-8f4d-3d558b3133f1,2019-01-29,1,0.0
27583,d2188b3c-f1af-4c78-b15a-a73bd73af034,2018-12-09,2,29.0
31902,ec202bcb-c786-4035-bd94-7d898aabe0dc,2019-02-01,2,51.0


## 0.1 Taking random 2000 users

In [10]:
df__time_series_events = df_front_back_combined.pivot(columns = 'timestamp',index = 'userid',values = 'event')
df__time_series_amount = df_front_back_combined.pivot(columns = 'timestamp',index = 'userid',values = 'amount')
df__time_series_events.to_csv(transformed_analyzed_events)
df__time_series_amount.to_csv(transformed_analyzed_amount)

## 1. RFM-analysis

In [11]:
df_recency =  pd.DataFrame(columns = df__time_series_events.columns)
df_monetary =  pd.DataFrame(columns = df__time_series_amount.columns)
df_frequency = pd.DataFrame(columns = df__time_series_events.columns)

In [ ]:
"""
The dataframe should have such columns: id of consumer, other columns as a time series
"""
for index, row in df__time_series_events.iterrows():
    arr = []
    arr = list(row)
    rec_arr = []
    freq_arr = []
    start_date = row.index[0]
    last_purchase = np.nan
    first_purchase = -1 #flag
    total_purchases = 0
    #RECENCY Customer did a explicit activity, such as renting cores or releasing cores.
    for j, entry in enumerate(arr):
        if (np.isnan(entry) or entry==0):            
            if (first_purchase==-1):
                rec_arr.append(row.index[j]-start_date)
            else:
                rec_arr.append(row.index[j]-last_purchase)
        else:
            #not nan
            total_purchases+=1
            if (first_purchase==-1):
                first_purchase = j
            rec_arr.append(0)
            last_purchase = row.index[j]
        #FREQUENCY it can be a cumulative average, the number of events divided by the number of days that have passed
        # since first purchase
        freq_arr.append(total_purchases/(j+1-first_purchase))
    df_recency.loc[index] = rec_arr
    df_frequency.loc[index]=freq_arr
df_recency=df_recency.applymap( lambda x:x if (type(x)==int) else x.days)

for index, row in df__time_series_amount.iterrows():
    arr = []
    arr = list(row)
    mon_arr = []
    #MONETARY  Moving average monetary value
    total_amount = 0 
    first_purchase = -1 #flag
    for j,entry in enumerate(arr):
        if (np.isnan(entry) or entry==0):
            total_amount = total_amount
        else:
            total_amount+=int(entry)
            if (first_purchase==-1):
                first_purchase = j
        mon_arr.append(total_amount/(j+1-first_purchase))
    df_monetary.loc[index] = mon_arr


## 2. Clustering

In [ ]:
ksh_monetary = KShape(n_clusters=4)
df_monetary['label'] = ksh_monetary.fit(df_monetary.values)
ksh_frequency = KShape(n_clusters=4)
df_frequency['label'] = ksh_frequency.fit(df_frequency.values)
ksh_recency = KShape(n_clusters=4)
df_recency['label'] = ksh_recency.fit(df_recency.values)

In [ ]:
df_monetary['label'] =ksh_monetary.labels_
df_frequency['label'] =ksh_frequency.labels_
df_recency['label'] =ksh_recency.labels_

In [ ]:
df_centroids = pd.DataFrame(columns = df_recency.drop(columns = 'label').columns)
for i in range(ksh_recency.cluster_centers_.shape[0]):
    df_centroids.loc['Centroid_for_recency_'+str(i)]= ksh_recency.cluster_centers_[i].ravel()
    df_centroids.loc['Centroid_for_frequency_'+str(i)]= ksh_frequency.cluster_centers_[i].ravel()
    df_centroids.loc['Centroid_for_monetary_'+str(i)]= ksh_monetary.cluster_centers_[i].ravel()

In [ ]:
df_monetary.to_csv(monetary_time_series)
df_frequency.to_csv(frequency_time_series)
df_recency.to_csv(recency_time_series)
df_centroids.to_csv(cluster_centroids_time_series)

## 3. Preprocessing for prediction

In [4]:
df = pd.read_csv(transformed_analyzed_amount)
df.head()

,userid,2018-07-05,2018-07-06,2018-07-08,2018-07-09,2018-07-10,2018-07-11,2018-07-12,2018-07-13,2018-07-14,...,2019-02-23,2019-02-24,2019-02-25,2019-02-26,2019-02-27,2019-02-28,2019-03-01,2019-03-02,2019-03-03,2019-03-04
0,0001c752-8daf-424d-af34-574f3ba624bb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0043a6cb-6314-4844-bd20-06c41915d398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00653016-d177-4080-86e6-4c69b865cb02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,169.0,0.0,0.0,0.0,NaN,NaN,NaN,157.0,0.0
3,007fb42f-93d4-4ad2-b595-5b773bda6e7d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,22.0,NaN,NaN,NaN,NaN,0.0
4,0094ec82-5fab-4343-9a96-a1bbdbad6cab,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = pd.melt(df,id_vars = ['userid'],var_name ='timestamp' ,value_name='value')
df.head()

,userid,timestamp,value
0,0001c752-8daf-424d-af34-574f3ba624bb,2018-07-05,NaN
1,0043a6cb-6314-4844-bd20-06c41915d398,2018-07-05,NaN
2,00653016-d177-4080-86e6-4c69b865cb02,2018-07-05,NaN
3,007fb42f-93d4-4ad2-b595-5b773bda6e7d,2018-07-05,NaN
4,0094ec82-5fab-4343-9a96-a1bbdbad6cab,2018-07-05,NaN


In [4]:
df_freq = pd.read_csv(frequency_time_series).loc[:,['Unnamed: 0','label']]
df_monet = pd.read_csv(monetary_time_series).loc[:,['Unnamed: 0','label']]
df_recency = pd.read_csv(recency_time_series).loc[:,['Unnamed: 0','label']]

In [5]:
df_monet.head()

,Unnamed: 0,label
0,0001c752-8daf-424d-af34-574f3ba624bb,2
1,0043a6cb-6314-4844-bd20-06c41915d398,0
2,00653016-d177-4080-86e6-4c69b865cb02,2
3,007fb42f-93d4-4ad2-b595-5b773bda6e7d,0
4,0094ec82-5fab-4343-9a96-a1bbdbad6cab,0


In [6]:
df_combined = df.merge(df_recency,left_on = 'userid',right_on='Unnamed: 0') \
    .merge(df_freq,left_on = 'userid',right_on='Unnamed: 0') \
    .merge(df_monet,left_on = 'userid',right_on='Unnamed: 0') \
    .drop(columns =['Unnamed: 0_x','Unnamed: 0_y','Unnamed: 0'])
df_combined.columns = ['id','timestamp','value','recency','frequency','monetary']
df_combined.sample(5)

,id,timestamp,value,recency,frequency,monetary
377237,d1c54acb-410d-42c3-b430-c4bcd124b532,2019-01-23,NaN,3,1,0
63180,260e012d-3ff5-4b8e-86a1-5d79a8616f79,2018-07-24,NaN,3,1,2
404254,e208e945-a37b-417a-ad99-c863f3f8073f,2018-10-28,NaN,3,0,0
325058,b16cd07f-c23b-49f5-8671-f5155c68dcf8,2018-08-27,NaN,3,1,2
173182,612b460e-58da-49c5-a0e3-264c7624b31c,2018-12-05,NaN,3,2,2


In [7]:
# STARTUP: change hash to real id
df_user = pd.read_csv(import_userdata).drop(columns =['Unnamed: 0','supplier','country']).reset_index()
df_user.head()
df_combined = df_combined.merge(df_user, left_on = 'id',right_on='id').drop(columns = 'id')
df_combined.sample(5)

,timestamp,value,recency,frequency,monetary,index
130914,2019-02-25,NaN,0,0,0,3297
221861,2019-01-11,NaN,0,0,0,1351
3412,2018-07-30,NaN,3,3,2,3965
168148,2019-01-22,NaN,1,0,2,1354
306980,2018-11-07,NaN,1,1,2,5705


In [8]:
df_combined['timestamp'] = df_combined['timestamp'].apply(lambda x:datetime.datetime.strptime(x, "%Y-%m-%d"))
df_combined.sample(2)

,timestamp,value,recency,frequency,monetary,index
340801,2018-09-09,NaN,0,2,0,2240
422049,2018-07-06,NaN,0,1,0,3180


In [9]:
df_combined = df_combined.fillna(0)

In [10]:
# number of months
test_period = 2
current_date = df_combined.timestamp.max()
df_test = df_combined[df_combined.timestamp >= current_date - pd.DateOffset(months = test_period )]
df_train = df_combined[df_combined.timestamp < current_date - pd.DateOffset(months = test_period )]

In [11]:
df_test.shape[0]/df_train.shape[0]

0.32967032967032966

In [12]:
y_train = df_train['value']
y_test = df_test['value']

X_train = df_train.drop('value', axis = 1)
X_test = df_test.drop('value', axis = 1)

## 4. Prediction

In [13]:
start_time = time.time()

cv = KFold(n_splits = 3, shuffle = True, random_state = 42)  

model = CatBoostRegressor(verbose = False, 
                           random_state=42)

param_grid = {
              'n_estimators': [200, 300],
              'max_depth': [3, 4], 
              'learning_rate': [0.03, 0.05, 0.07] 
             }

gs_cb = GridSearchCV(model, 
                  param_grid,
                  cv = cv,
                  scoring = 'neg_mean_squared_error')

gs_cb.fit(X_train, y_train)

#train_RMSE = np.sqrt(-gs_cb.cv_results_['mean_train_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()
#train_std = np.sqrt(gs_cb.cv_results_['std_train_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()

cv_RMSE = np.sqrt(-gs_cb.cv_results_['mean_test_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()
cv_std = np.sqrt(gs_cb.cv_results_['std_test_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()

test_RMSE = np.sqrt(mean_squared_error(gs_cb.predict(X_test), y_test))

#print('train RMSE: {} +/- {}'.format(np.round(train_RMSE, 2), np.round(train_std, 2)))
print('CV RMSE: {} +/- {}'.format(np.round(cv_RMSE, 2), np.round(cv_std, 2)))
print('test RMSE: {}'.format(np.round(test_RMSE, 2)))
print(gs_cb.best_params_)
print('time :{} min'.format(np.round((time.time() - start_time)/60, 2))) 

filename = 'CatBoost_startup.sav'
pickle.dump(gs_cb, open(filename, 'wb'))

CV RMSE: 20.44 +/- 7.25
test RMSE: 155.43
{'learning_rate': 0.07, 'max_depth': 4, 'n_estimators': 300}
time :7.93 min


In [14]:
result = X_test
result['predicted_value']=gs_cb.predict(X_test)
result['true_value']=y_test


result.to_csv(prediction_path)